In [222]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [158]:
# ナップサック問題を作る関数
def generate_problem(N, RAND_MAX):
    value = np.random.randint(1, RAND_MAX, N)
    weight = np.random.randint(1, RAND_MAX, N)
    ratio = value / weight
    capacity = N * RAND_MAX / 6
    return [value, weight, ratio, capacity]

In [159]:
def binary(i, X, res):
    if i >= N:
        if weight @ X <= capacity:
            return [value @ X, np.array(X)]
    else:
        X[i] = 0
        res += binary(i+1, X, [])
        X[i] = 1
        res += binary(i+1, X, [])
    return res

In [160]:
def rec(i, j):
    # 全ての品物を見終わった
    print("*")
    if i == n:
        res = 0
    
    # 品物が入らないのでスルー
    elif j < w[i]:
        res = rec(i+1, j)
    else:
        # iを入れないでスルーする場合
        # iを入れて、重さを減らして、価値を増やす場合
        # 両者のmaxをとる
        res = max(rec(i+1, j), rec(i+1, j-w[i]) + v[i])

    return res

In [193]:
# 1から0の順番にした方が無駄な探索が入らないのか??
def binary_prued(i, X, res):    
    # 分枝限定(暫定値以上の解がない)
    if (value[:i] @ X[:i]) + (value[i:] @ [1 for i in range(N-i)]) < max_value:
        return [0, None]
    
    # 分枝限定(許容解)
    if weight[:i] @ X[:i] > capacity:
        return [0, None]
    
    # 荷物を全部見た
    if i >= N:
        if weight @ X <= capacity:
            if value @ X > max_value[0]:
                max_value[0] = value @ X
            return [value @ X, np.array(X)]
    
    else:
        X[i] = 0
        res += binary_prued_(i+1, X, [])
        X[i] = 1
        res += binary_prued_(i+1, X, [])
    return res

In [194]:
# 1から0の順番にした方が無駄な探索が入らないのか??
def binary_prued_(i, X, res):    
    # 分枝限定(暫定値以上の解がない)
    if (value[:i] @ X[:i]) + (value[i:] @ [1 for i in range(N-i)]) < max_value:
        return [0, None]
    
    # 分枝限定(許容解)
    if weight[:i] @ X[:i] > capacity:
        return [0, None]
    
    # 荷物を全部見た
    if i >= N:
        if weight @ X <= capacity:
            if value @ X > max_value[0]:
                max_value[0] = value @ X
            return [value @ X, np.array(X)]
    
    else:
        X[i] = 1
        res += binary_prued_(i+1, X, [])
        X[i] = 0
        res += binary_prued_(i+1, X, [])
    return res

In [195]:
def prued_solve():
    global max_value
    max_value[0] = 0
    enum = np.array(binary_prued(0, np.array([0 for i in range(N)]), [])).reshape(-1,2)
    enum_T = enum.T
    return enum[np.argsort(enum_T[0])[-1]]

In [196]:
def prued_solve_():
    global max_value
    max_value[0] = 0
    enum = np.array(binary_prued_(0, np.array([0 for i in range(N)]), [])).reshape(-1,2)
    enum_T = enum.T
    return enum[np.argsort(enum_T[0])[-1]]

In [173]:
def enumeration():
    enum = np.array(binary(0, np.array([0 for i in range(N)]), [])).reshape(-1,2)
    enum_T = enum.T
    return enum[np.argsort(enum_T[0])[-1]]

In [174]:
def greedy():
    ratio_index = np.argsort(ratio)[::-1]
    w = 0
    v = 0
    i = 0
    ans = [0 for i in range(N)]
    while i < N:
        if w + weight[ratio_index[i]] <= capacity:
            w += weight[ratio_index[i]]
            ans[ratio_index[i]] = 1
            v += value[ratio_index[i]]
            i += 1
        else:
            i += 1
    return [v,ans]

In [175]:
def relaxation():
    ratio_index = np.argsort(ratio)[::-1]
    w = 0
    v = 0
    i = 0
    ans = [0 for i in range(N)]
    while i < N:
        if w + weight[ratio_index[i]] <= capacity:
            w += weight[ratio_index[i]]
            ans[ratio_index[i]] = 1
            v += value[ratio_index[i]]
            i += 1
        else:
            ans[ratio_index[i]] = (capacity - w) / weight[ratio_index[i]]
            v += ((capacity - w) / weight[ratio_index[i]]) * value[ratio_index[i]]
            break
    
    return [v, ans]

In [209]:
def fix_greedy(index):
    ratio_index = np.argsort(ratio)[::-1]
    w = weight[index]
    v = value[index]
    i = 1
    ans = [1 if i == index else 0 for i in range(N)]
    
    while i < N-1:
        if i != index:
            if w + weight[ratio_index[i]] <= capacity:
                w += weight[ratio_index[i]]
                ans[ratio_index[i]] = 1
                v += value[ratio_index[i]]
                i += 1
            else:
                i += 1
    return [v,ans] 

In [213]:
def half_approx():
    # 貪欲
    greedy_ans = greedy()
    max_ef = 0
    max_ef_index = False
    for i in greedy_ans[1]:
        if i == 0:
            if (max_ef <= ratio[i]) and (weight[i] <= capacity):
                max_ef = ratio[i]
                max_ef_index = i
    
    fix_greedy_ans = fix_greedy(max_ef_index)
    
    if greedy_ans[0] < fix_greedy_ans[0]:
        return fix_greedy_ans
    else:
        return greedy_ans

In [214]:
def half_approx_test():
    # 貪欲
    greedy_ans = greedy()
    max_ef = 0
    max_ef_index = False
    for i in greedy_ans[1]:
        if i == 0:
            if max_ef <= ratio[i]:
                max_ef = ratio[i]
                max_ef_index = i
    
    fix_greedy_ans = fix_greedy(max_ef_index)
    
    return greedy_ans, fix_greedy_ans